# Detección de Intrusiones y Análisis de Anomalías en Tráfico de Red mediante Técnicas Estadísticas

**Universidad de La Habana, MATCOM**  
**Curso:** Estadística 2025-2026  
**Proyecto Final:** Análisis Estadístico Aplicado a Seguridad Informática

---

## 1. Introducción al Proyecto

### 1.1 Contexto y Motivación

La seguridad informática es uno de los pilares fundamentales en la infraestructura tecnológica moderna. Los Sistemas de Detección de Intrusiones (IDS) tradicionales, basados en firmas conocidas, presentan limitaciones significativas frente a ataques emergentes o modificados (zero-day attacks). 

Este proyecto propone un enfoque complementario basado en **análisis estadístico del comportamiento del tráfico de red**, permitiendo identificar patrones anómalos sin depender exclusivamente de firmas previamente catalogadas. Este tipo de aproximación resulta especialmente relevante en entornos dinámicos donde los ataques evolucionan constantemente.

### 1.2 Dataset: NSL-KDD

**Fuente:** [NSL-KDD en Kaggle](https://www.kaggle.com/datasets/hassan06/nslkdd)

El dataset NSL-KDD es una versión refinada del clásico KDD Cup 1999, diseñada específicamente para eliminar redundancias y sesgos presentes en el conjunto original. Es ampliamente reconocido como estándar académico para la evaluación de algoritmos de detección de intrusiones.

**Características principales:**
- **41 variables predictoras** + 1 variable objetivo (`attack_type`) + 1 nivel de dificultad
- **Tipos de ataques:** Normal, DoS (Denial of Service), Probe (escaneo/sondeo), R2L (Remote to Local), U2R (User to Root)
- **Conjunto de entrenamiento:** 25,192 observaciones
- **Conjunto de prueba:** 22,544 observaciones

**Categorización de variables:**
- **Básicas:** Derivadas de cabeceras TCP/IP (duration, protocol_type, src_bytes, dst_bytes, flag)
- **De contenido:** Información sobre el payload (num_failed_logins, root_shell, etc.)
- **De tráfico:** Estadísticas temporales orientadas a detectar patrones (count, serror_rate, etc.)

---

### 1.3 Carga de Datos y Preparación Inicial del Dataset NSL-KDD


#### Imports y Configuración

In [ ]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import sklearn as sk
import scipy
import os
import statsmodels

# Configuración de visualizaciones
warnings.filterwarnings('ignore')
%matplotlib inline

# Estilo global para mantener consistencia en todas las visualizaciones
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.titlesize'] = 13
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10

# Paleta de colores consistente para categorías de ataque
# Se utilizará en todas las visualizaciones del proyecto
attack_colors = {
    'Normal': '#2ecc71',    # Verde - Tráfico legítimo
    'DoS': '#e74c3c',       # Rojo - Ataques de denegación de servicio
    'Probe': '#f39c12',     # Naranja - Ataques de reconocimiento
    'R2L': '#9b59b6',       # Morado - Acceso remoto no autorizado
    'U2R': '#34495e'        # Gris oscuro - Escalada de privilegios
}

print("✅ Librerías importadas y configuración de visualización establecida. \n Se utilizan bibliotecas estándar de análisis de datos y visualización ampliamente aceptadas en ciencia de datos, lo que garantiza reproducibilidad y compatibilidad.")

#### Carga de Datos

In [ ]:
# Definir los nombres de las columnas (43 columnas en total)
col_names = [
    "duration", "protocol_type", "service", "flag", "src_bytes",
    "dst_bytes", "land", "wrong_fragment", "urgent", "hot",
    "num_failed_logins", "logged_in", "num_compromised", "root_shell",
    "su_attempted", "num_root", "num_file_creations", "num_shells",
    "num_access_files", "num_outbound_cmds", "is_host_login",
    "is_guest_login", "count", "srv_count", "serror_rate",
    "srv_serror_rate", "rerror_rate", "srv_rerror_rate", "same_srv_rate",
    "diff_srv_rate", "srv_diff_host_rate", "dst_host_count",
    "dst_host_srv_count", "dst_host_same_srv_rate",
    "dst_host_diff_srv_rate", "dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate", "dst_host_serror_rate",
    "dst_host_srv_serror_rate", "dst_host_rerror_rate",
    "dst_host_srv_rerror_rate", "attack_type", "difficulty_level"
]

# Cargar los datasets
# Nota: Ajusta las rutas según tu estructura de carpetas
train_df = pd.read_csv('Data/KDDTrain+_20Percent.txt', 
                       names=col_names, 
                       header=None)

test_df = pd.read_csv('Data/KDDTest+.txt', 
                      names=col_names, 
                      header=None)

# Crear variable binaria para clasificación binaria (Normal vs. Ataque)
train_df['is_attack'] = (train_df['attack_type'] != 'normal').astype(int)
test_df['is_attack'] = (test_df['attack_type'] != 'normal').astype(int)

# Mostrar información básica
print(f"📊 Datos de entrenamiento: {train_df.shape}")
print(f"📊 Datos de prueba: {test_df.shape}")
print(f"\n✅ Datasets cargados exitosamente")


### 1.4 Mapeo de Categorías de Ataque

El dataset NSL-KDD contiene 40 tipos de ataques específicos que se agrupan en 4 categorías principales más la clase normal. A continuación se realiza el mapeo oficial según la documentación del Canadian Institute for Cybersecurity:

- **Normal:** Tráfico de red legítimo
- **DoS (Denial of Service):** Ataques que buscan denegar el servicio mediante sobrecarga de recursos
- **Probe (Probing/Scanning):** Ataques de reconocimiento que escanean la red en busca de vulnerabilidades
- **R2L (Remote to Local):** Intentos de acceso no autorizado desde una máquina remota
- **U2R (User to Root):** Intentos de escalada de privilegios de usuario normal a superusuario

Este agrupamiento reduce la complejidad del problema y permite realizar inferencias estadísticas más robustas sobre patrones generales de ataque.

In [ ]:
# Diccionario oficial de mapeo de ataques específicos a categorías generales
# Fuente: Documentación oficial NSL-KDD (Canadian Institute for Cybersecurity)
attack_category_mapping = {
    # Tráfico Normal
    'normal': 'Normal',
    
    # DoS (Denial of Service) - Ataques de denegación de servicio
    'back': 'DoS', 'land': 'DoS', 'neptune': 'DoS', 'pod': 'DoS',
    'smurf': 'DoS', 'teardrop': 'DoS', 'mailbomb': 'DoS', 'apache2': 'DoS',
    'processtable': 'DoS', 'udpstorm': 'DoS',
    
    # Probe (Probing/Scanning) - Ataques de reconocimiento
    'ipsweep': 'Probe', 'nmap': 'Probe', 'portsweep': 'Probe',
    'satan': 'Probe', 'mscan': 'Probe', 'saint': 'Probe',
    
    # R2L (Remote to Local) - Acceso no autorizado desde máquina remota
    'ftp_write': 'R2L', 'guess_passwd': 'R2L', 'imap': 'R2L',
    'multihop': 'R2L', 'phf': 'R2L', 'spy': 'R2L',
    'warezclient': 'R2L', 'warezmaster': 'R2L', 'sendmail': 'R2L',
    'named': 'R2L', 'snmpgetattack': 'R2L', 'snmpguess': 'R2L',
    'xlock': 'R2L', 'xsnoop': 'R2L', 'worm': 'R2L', 'xterm': 'R2L', 
    
    # U2R (User to Root) - Escalada de privilegios
    'buffer_overflow': 'U2R', 'loadmodule': 'U2R', 'perl': 'U2R',
    'rootkit': 'U2R', 'httptunnel': 'U2R', 'ps': 'U2R', 'sqlattack': 'U2R'
}

# Aplicar mapeo a ambos datasets
train_df['attack_category'] = train_df['attack_type'].map(attack_category_mapping)
test_df['attack_category'] = test_df['attack_type'].map(attack_category_mapping)

# Verificar que no hay valores sin mapear
print("🔍 Verificación de mapeo de categorías:")
unmapped_train = train_df['attack_category'].isna().sum()
unmapped_test = test_df['attack_category'].isna().sum()
print(f"   Valores sin mapear en train: {unmapped_train}")
print(f"   Valores sin mapear en test: {unmapped_test}")

if unmapped_test > 0:
    print(f"\n⚠️ ADVERTENCIA: Hay {unmapped_test} ataques en test sin categoría asignada.")

# Crear orden categórico para visualizaciones consistentes
category_order = ['Normal', 'DoS', 'Probe', 'R2L', 'U2R']
train_df['attack_category'] = pd.Categorical(
    train_df['attack_category'], 
    categories=category_order, 
    ordered=True
)
test_df['attack_category'] = pd.Categorical(
    test_df['attack_category'], 
    categories=category_order, 
    ordered=True
)

# Mostrar distribución de categorías
print("\n📊 Distribución de categorías en ENTRENAMIENTO:")
category_dist_train = train_df['attack_category'].value_counts()
category_pct_train = train_df['attack_category'].value_counts(normalize=True) * 100
category_summary_train = pd.DataFrame({
    'Frecuencia': category_dist_train,
    'Porcentaje': category_pct_train.round(2)
})
print(category_summary_train)

print("\n📊 Distribución de categorías en PRUEBA:")
category_dist_test = test_df[test_df['attack_category'].notna()]['attack_category'].value_counts()
category_pct_test = test_df[test_df['attack_category'].notna()]['attack_category'].value_counts(normalize=True) * 100
category_summary_test = pd.DataFrame({
    'Frecuencia': category_dist_test,
    'Porcentaje': category_pct_test.round(2)
})
print(category_summary_test)

print("\n✅ Mapeo de categorías completado exitosamente")

### 1.4 Preparación de Muestra Estratificada para Visualizaciones

Para optimizar el rendimiento de visualizaciones complejas (como scatterplot matrices y pairplots), crearemos una muestra estratificada de 5,000 observaciones que mantenga las proporciones originales de cada categoría de ataque. 

**Nota importante:** Esta muestra se utilizará **exclusivamente para visualizaciones**. Todos los análisis estadísticos (correlaciones, pruebas de hipótesis, modelos) se realizarán sobre el dataset completo.

In [ ]:
# Configuración del muestreo estratificado
sample_size = 5000
random_state = 42  # Para reproducibilidad

# Crear muestra estratificada manteniendo proporciones de cada categoría
train_sample = train_df.groupby('attack_category', group_keys=False).apply(
    lambda x: x.sample(
        n=min(len(x), int(sample_size * len(x) / len(train_df))),
        random_state=random_state
    )
).reset_index(drop=True)

print(f"📊 Muestra estratificada creada: {len(train_sample):,} observaciones")
print(f"\n✅ Verificación de estratificación:")

# Comparar proporciones originales vs. muestra
comparison = pd.DataFrame({
    'Original (%)': train_df['attack_category'].value_counts(normalize=True).sort_index() * 100,
    'Muestra (%)': train_sample['attack_category'].value_counts(normalize=True).sort_index() * 100
})
comparison['Diferencia (pp)'] = (comparison['Muestra (%)'] - comparison['Original (%)']).abs()
print(comparison.round(2))

print("\n💡 Esta muestra se usará únicamente para visualizaciones pesadas.")
print("   Los análisis estadísticos utilizarán el dataset completo.")